In [1]:
import os
import sys
import pickle
import time
import librosa
import torch
import numpy as np
import pandas as pd
import scipy
from tqdm import tqdm

#sys.path.append("./utils/")
import laugh_segmenter
import models, configs

from utils import dataset_utils, audio_utils, data_loaders, torch_utils
from torch import optim, nn
from functools import partial
from distutils.util import strtobool

sample_rate = 8000

def segment_laughter(
    input_audio_file,
    model_path="checkpoints/in_use/resnet_with_augmentation",
    config="resnet_with_augmentation",
    threshold=0.5,
    min_length=0.2,
    output_dir=None
):
    device = torch.device("cpu")
    print(f"Using device {device}")

    ##### Load the Model
    config_data = configs.CONFIG_MAP[config]
    model = config_data["model"](
        dropout_rate=0.0,
        linear_layer_size=config_data["linear_layer_size"],
        filter_sizes=config_data["filter_sizes"],
    )
    feature_fn = config_data["feature_fn"]
    model.set_device(device)

    if os.path.exists(model_path):
        torch_utils.load_checkpoint(model_path + "/best.pth.tar", model)
        model.eval()
    else:
        raise Exception(f"Model checkpoint not found at {model_path}")

    ##### Load the audio file and features
    inference_dataset = data_loaders.SwitchBoardLaughterInferenceDataset(
        audio_path=input_audio_file, feature_fn=feature_fn, sr=sample_rate
    )

    collate_fn = partial(
        audio_utils.pad_sequences_with_labels,
        expand_channel_dim=config_data["expand_channel_dim"],
    )

    inference_generator = torch.utils.data.DataLoader(
        inference_dataset,
        num_workers=4,
        batch_size=8,
        shuffle=False,
        collate_fn=collate_fn,
    )

    ##### Make Predictions
    probs = []
    for model_inputs, _ in tqdm(inference_generator):
        x = torch.from_numpy(model_inputs).float().to(device)
        preds = model(x).cpu().detach().numpy().squeeze()
        if len(preds.shape) == 0:
            preds = [float(preds)]
        else:
            preds = list(preds)
        probs += preds
    probs = np.array(probs)

    file_length = audio_utils.get_audio_length(input_audio_file)

    fps = len(probs) / float(file_length)

    probs = laugh_segmenter.lowpass(probs)
    instances = laugh_segmenter.get_laughter_instances(
        probs, threshold=threshold, min_length=min_length, fps=fps
    )

    results = []

    if len(instances) > 0:
        full_res_y, full_res_sr = librosa.load(input_audio_file, sr=44100)
        wav_paths = []
        maxv = np.iinfo(np.int16).max

        base_name = os.path.splitext(os.path.basename(input_audio_file))[0]
        
        if output_dir is None:
            raise Exception("Need to specify an output directory to save audio files")
        else:
            os.system(f"mkdir -p {output_dir}")
            for index, instance in enumerate(instances):
                laughs = laugh_segmenter.cut_laughter_segments(
                    [instance], full_res_y, full_res_sr
                )
                wav_path = os.path.join(output_dir, f"{base_name}_laugh_{index}.wav")
                scipy.io.wavfile.write(
                    wav_path, full_res_sr, (laughs * maxv).astype(np.int16)
                )
                results.append(
                    {
                        "file_name": wav_path,
                        "start": instance[0],
                        "end": instance[1],
                    }
                )

    return results


AttributeError: module 'numpy' has no attribute 'complex'.
`np.complex` was a deprecated alias for the builtin `complex`. To avoid this error in existing code, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
# This used to be the original way of doing things
# python segment_laughter.py --input_audio_file=./input_audio/test_2.mp3 --output_dir=./tst_wave --save_to_textgrid=False --save_to_audio_files=True --min_length=0.2 --threshold=0.5

In [2]:
laughs =segment_laughter(
    input_audio_file="./input_audio" + "/joke1.1_5dcc826c-e70a-4c28-bd4e-dca0e9023fa8_1560890971288_431.mp3",
    output_dir="./test",
)

Using device cpu
training with dropout=0.0
Loading checkpoint at: checkpoints/in_use/resnet_with_augmentation/best.pth.tar
Loading checkpoint at step:  60600


100%|██████████| 67/67 [00:41<00:00,  1.62it/s]


In [3]:
laughs

[{'file_name': './test/joke1.1_5dcc826c-e70a-4c28-bd4e-dca0e9023fa8_1560890971288_431_laugh_0.wav',
  'start': 5.677570093457945,
  'end': 7.973831775700935},
 {'file_name': './test/joke1.1_5dcc826c-e70a-4c28-bd4e-dca0e9023fa8_1560890971288_431_laugh_1.wav',
  'start': 12.742990654205608,
  'end': 13.474766355140188}]

In [10]:
laughs_detected

,filename,num_laugh,results
0,joke2.1_5ce0c525-1675-4a81-a8ce-17adbd0fd06a_1...,0,[]
1,joke5.3_f4e620b4-8cf1-485a-961b-3a0828415ec5_1...,1,[{'file_name': './output_segments/joke5.3_f4e6...
2,joke1.1_5dcc826c-e70a-4c28-bd4e-dca0e9023fa8_1...,2,[{'file_name': './output_segments/joke1.1_5dcc...


In [ ]:
def filter_file_names(file_list, partial_list):
    filtered_files = []
    
    for file_name in file_list:
        if file_name.startswith("joke1.1"):
            for partial_name in partial_list:
                if partial_name in file_name:
                    filtered_files.append(file_name)
                    break  # If there's a match, no need to check other partial names
    
    return filtered_files

# Example usage:
file_list = [
    "joke1.1_0d3cc120-8440-4579-9fd7-e7b88eb9a45f_1561401565511_636.mp3",
    "joke1.1_12345-67890-abcdef_1561401565511_636.mp3",
    "other_file.mp3",
]

partial_list = [
    "4579-9fd7-e7b88eb9a45f",
    "abcdef",
]

filtered_files = filter_file_names(file_list, partial_list)
print(filtered_files)


In [36]:
import os
import pandas as pd

def process_audio_files(filtered_file_list=None):
    # Define the input directory and output directory
    input_dir = "./input_audio/"
    output_dir = "./output_segments/"
    
    if not filtered_file_list:
        # List all files in the input directory
        all_files = os.listdir(input_dir)
        
        # Filter out only the MP3 files
        mp3_files = [f for f in all_files if f.endswith('.mp3')]
    else:
        mp3_files = filtered_file_list
    
    # Initialize an empty list to store results
    results = []
    
    # Process each MP3 file
    for mp3_file in mp3_files:
        # Get the full path of the MP3 file
        full_path = os.path.join(input_dir, mp3_file)
        
        # Run the segment_laughter function
        laughs = segment_laughter(input_audio_file=full_path, output_dir=output_dir)
        
        # Append the results to the results list
        results.append({"filename": mp3_file, "num_laughs":len(laughs), "results": laughs})
    
    # Convert the results list to a pandas DataFrame
    df = pd.DataFrame(results)
    
    return df


In [14]:
# Call the process_audio_files function
df = process_audio_files()


Using device cpu
training with dropout=0.0
Loading checkpoint at: checkpoints/in_use/resnet_with_augmentation/best.pth.tar
Loading checkpoint at step:  60600


  7%|▋         | 12/176 [00:09<02:03,  1.33it/s]


KeyboardInterrupt: 

In [4]:
df

,filename,num_laughs,results
0,joke2.1_5ce0c525-1675-4a81-a8ce-17adbd0fd06a_1...,0,[]
1,joke5.3_f4e620b4-8cf1-485a-961b-3a0828415ec5_1...,1,[{'file_name': './output_segments/joke5.3_f4e6...
2,joke1.1_5dcc826c-e70a-4c28-bd4e-dca0e9023fa8_1...,2,[{'file_name': './output_segments/joke1.1_5dcc...


In [8]:
for results in df.results.values:
    print(results)

[]
[{'file_name': './output_segments/joke5.3_f4e620b4-8cf1-485a-961b-3a0828415ec5_1577919787295_189_laugh_0.wav', 'start': 5.915014164305949, 'end': 12.81586402266289}]
[{'file_name': './output_segments/joke1.1_5dcc826c-e70a-4c28-bd4e-dca0e9023fa8_1560890971288_431_laugh_0.wav', 'start': 5.677570093457945, 'end': 7.973831775700935}, {'file_name': './output_segments/joke1.1_5dcc826c-e70a-4c28-bd4e-dca0e9023fa8_1560890971288_431_laugh_1.wav', 'start': 12.742990654205608, 'end': 13.474766355140188}]


In [9]:
{'file_name': './output_segments/joke5.3_f4e620b4-8cf1-485a-961b-3a0828415ec5_1577919787295_189_laugh_0.wav', 'start': 5.915014164305949, 'end': 12.81586402266289}

{'file_name': './output_segments/joke5.3_f4e620b4-8cf1-485a-961b-3a0828415ec5_1577919787295_189_laugh_0.wav',
 'start': 5.915014164305949,
 'end': 12.81586402266289}

In [10]:
{'file_name': './output_segments/joke1.1_5dcc826c-e70a-4c28-bd4e-dca0e9023fa8_1560890971288_431_laugh_0.wav', 'start': 5.677570093457945, 'end': 7.973831775700935}

{'file_name': './output_segments/joke1.1_5dcc826c-e70a-4c28-bd4e-dca0e9023fa8_1560890971288_431_laugh_0.wav',
 'start': 5.677570093457945,
 'end': 7.973831775700935}

In [11]:
{'file_name': './output_segments/joke1.1_5dcc826c-e70a-4c28-bd4e-dca0e9023fa8_1560890971288_431_laugh_1.wav', 'start': 12.742990654205608, 'end': 13.474766355140188}

{'file_name': './output_segments/joke1.1_5dcc826c-e70a-4c28-bd4e-dca0e9023fa8_1560890971288_431_laugh_1.wav',
 'start': 12.742990654205608,
 'end': 13.474766355140188}

In [16]:
excel_file  = pd.read_excel("./baby_joke_1_laugh_1.xlsx")

In [17]:
excel_file.head()

,VIDEOID/RESPONSEID,ChildHashedID,age_in_days,gender,Consent,JokeOrder,joke1.laugh.1,joke1.howfunny.1,joke1.laugh.2,joke1.howfunny.2,...,ThatsNotACat.laugh.2,ThatsNotACat.howfunny.2,ThatsNotACat.laugh.3,ThatsNotACat.howfunny.3,TearingPaper.laugh.1,TearingPaper.howfunny.1,TearingPaper.laugh.2,TearingPaper.howfunny.2,TearingPaper.laugh.3,TearingPaper.howfunny.3
0,4d16ddd9-e207-45d0-bed8-4e2bf8df60ea,FHN26F,105,f,accepted,"['ThatsNotACat', 'Peekaboo', 'ThatsNotAHat', '...",No,Not funny,No,Not funny,...,No,Not funny,No,Not funny,No,Slightly funny,No,Not funny,No,Not funny
1,73e855f3-6c3c-4673-86a0-c7f3dcfc7531,ZCH3UU,121,m,accepted,"['TearingPaper', 'NomNomNom', 'ThatsNotAHat', ...",No,Not funny,No,Not funny,...,No,Not funny,No,Not funny,No,Not funny,No,Not funny,No,Not funny
2,f889fba2-ad55-4b6a-ba67-baf589bf562e,USMEcW,132,f,accepted,"['TearingPaper', 'Peekaboo', 'ThatsNotAHat', '...",No,Not funny,No,Not funny,...,No,Not funny,No,Not funny,No,Not funny,No,Not funny,No,Not funny
3,ffd96b9b-89f7-4664-89a9-368ddea2e3e1,ZWYN3M,135,f,accepted,"['ThatsNotACat', 'NomNomNom', 'ThatsNotAHat', ...",No,Funny,No,Funny,...,No,Funny,No,Funny,No,Not funny,No,Not funny,No,Slightly funny
4,b1fbc9a4-f81f-4926-bc8f-c69b249d3259,PYDMU3,138,f,accepted,"['TearingPaper', 'ThatsNotAHat', 'Peekaboo', '...",No,Slightly funny,No,Not funny,...,No,Not funny,No,Slightly funny,No,Slightly funny,No,Not funny,Yes,Slightly funny


In [20]:
excel_file['joke1.laugh.1'].value_counts()

No     58
Yes    35
Name: joke1.laugh.1, dtype: int64

In [30]:
labelled_joke1_1_partial_filename = excel_file['VIDEOID/RESPONSEID'].to_list()

In [31]:
len(labelled_joke1_1_partial_filename)

94

In [22]:
def filter_file_names(file_list, partial_list):
    filtered_files = []
    
    for file_name in file_list:
        if file_name.startswith("joke1.1"):
            for partial_name in partial_list:
                if partial_name in file_name:
                    filtered_files.append(file_name)
                    break  # If there's a match, no need to check other partial names
    
    return filtered_files



['joke1.1_0d3cc120-8440-4579-9fd7-e7b88eb9a45f_1561401565511_636.mp3', 'joke1.1_12345-67890-abcdef_1561401565511_636.mp3']


In [23]:
# Define the input directory and output directory
input_dir = "./input_audio/"

# List all files in the input directory
all_files = os.listdir(input_dir)

# Filter out only the MP3 files
mp3_files = [f for f in all_files if f.endswith('.mp3')]

In [24]:
len(mp3_files)

1538

In [25]:
mp3_files[:5]

['joke1.1_088ca5ae-9b47-469b-a197-d7872f5ae9d6_1559764135196_649.mp3',
 'joke3.3_20a5c54f-d1b0-48d7-bcad-b86d3f6c659e_1568872464025_665.mp3',
 'joke5.3_52d1e1ce-ab4b-4374-94ac-d21eacb0006a_1569452562666_970.mp3',
 'joke1.1_5f9cb3c3-dba6-4a27-9881-57a95c7efe7a_1587767083654_972.mp3',
 'joke1.1_22fa22ac-9f30-40e2-89b8-41ee3a8e89cf_1586730181497_917.mp3']

In [32]:
filtered_files = filter_file_names(mp3_files, labelled_joke1_1_partial_filename)


In [33]:
len(filtered_files)

119

In [34]:
filtered_files

['joke1.1_088ca5ae-9b47-469b-a197-d7872f5ae9d6_1559764135196_649.mp3',
 'joke1.1_5f9cb3c3-dba6-4a27-9881-57a95c7efe7a_1587767083654_972.mp3',
 'joke1.1_22fa22ac-9f30-40e2-89b8-41ee3a8e89cf_1586730181497_917.mp3',
 'joke1.1_2a3186e4-8f1e-42e8-84a3-172610e3dec4_1563800203941_506.mp3',
 'joke1.1_9f94b24b-25ac-4266-a3f5-17f48f7feaa6_1556056374180_338.mp3',
 'joke1.1_246e8874-e4dd-4fbb-83ce-41d70ca77876_1555356197789_437.mp3',
 'joke1.1_1319e4c3-e360-40f4-9279-fb1cfa9ed65a_1561383699061_749.mp3',
 'joke1.1_7d9817b4-3ee8-4cb3-a474-1789b08faf3a_1553875275933_594.mp3',
 'joke1.1_c4f82db5-2182-47c3-8d85-ac0ca5b205c4_1555968523070_953.mp3',
 'joke1.1_e986fa51-78fe-4e4e-99cb-a7540bd83174_1588650654970_398.mp3',
 'joke1.1_1319e4c3-e360-40f4-9279-fb1cfa9ed65a_1561383722009_464.mp3',
 'joke1.1_60048bfe-321a-4d65-b842-5bea0ed98c58_1585566015223_844.mp3',
 'joke1.1_f9ffff9f-7659-4b23-941e-988979f69ded_1579140202923_616.mp3',
 'joke1.1_5ce0c525-1675-4a81-a8ce-17adbd0fd06a_1553722447759_636.mp3',
 'joke

In [38]:
processed_results = process_audio_files(filtered_file_list=filtered_files)

Using device cpu
training with dropout=0.0
Loading checkpoint at: checkpoints/in_use/resnet_with_augmentation/best.pth.tar
Loading checkpoint at step:  60600


100%|██████████| 176/176 [01:49<00:00,  1.61it/s]


Using device cpu
training with dropout=0.0
Loading checkpoint at: checkpoints/in_use/resnet_with_augmentation/best.pth.tar
Loading checkpoint at step:  60600


100%|██████████| 92/92 [00:57<00:00,  1.61it/s]


Using device cpu
training with dropout=0.0
Loading checkpoint at: checkpoints/in_use/resnet_with_augmentation/best.pth.tar
Loading checkpoint at step:  60600


...

100%|██████████| 73/73 [00:48<00:00,  1.50it/s]


Using device cpu
training with dropout=0.0
Loading checkpoint at: checkpoints/in_use/resnet_with_augmentation/best.pth.tar
Loading checkpoint at step:  60600


100%|██████████| 114/114 [01:11<00:00,  1.59it/s]


Using device cpu
training with dropout=0.0
Loading checkpoint at: checkpoints/in_use/resnet_with_augmentation/best.pth.tar
Loading checkpoint at step:  60600


100%|██████████| 122/122 [01:15<00:00,  1.62it/s]


In [39]:
processed_results.head()

,filename,num_laughs,results
0,joke1.1_088ca5ae-9b47-469b-a197-d7872f5ae9d6_1...,0,[]
1,joke1.1_5f9cb3c3-dba6-4a27-9881-57a95c7efe7a_1...,2,[{'file_name': './output_segments/joke1.1_5f9c...
2,joke1.1_22fa22ac-9f30-40e2-89b8-41ee3a8e89cf_1...,0,[]
3,joke1.1_2a3186e4-8f1e-42e8-84a3-172610e3dec4_1...,1,[{'file_name': './output_segments/joke1.1_2a31...
4,joke1.1_9f94b24b-25ac-4266-a3f5-17f48f7feaa6_1...,1,[{'file_name': './output_segments/joke1.1_9f94...


In [40]:
processed_results.to_csv("joke_1_1_laughters.csv")

In [43]:
processed_results.to_excel( "joke_1_1_laughters.xlsx",header=True)

In [44]:
processed_results.filename.nunique()

119

In [45]:
len(filtered_files)

119

In [47]:
len(labelled_joke1_1_partial_filename)

94

In [48]:
def find_multiple_matches(partial_names, complete_names):
    for partial in partial_names:
        matches = [name for name in complete_names if partial in name]
        if len(matches) > 1:
            print(f"Partial name: {partial}")
            for match in matches:
                print(f"  - {match}")





In [49]:
find_multiple_matches(labelled_joke1_1_partial_filename, filtered_files)


Partial name: 4d16ddd9-e207-45d0-bed8-4e2bf8df60ea
  - joke1.1_4d16ddd9-e207-45d0-bed8-4e2bf8df60ea_1587233794827_922.mp3
  - joke1.1_4d16ddd9-e207-45d0-bed8-4e2bf8df60ea_1587233759265_222.mp3
Partial name: dfbe3248-21ac-49d9-8da0-fea5314856b1
  - joke1.1_dfbe3248-21ac-49d9-8da0-fea5314856b1_1585326431489_495.mp3
  - joke1.1_dfbe3248-21ac-49d9-8da0-fea5314856b1_1585326472515_357.mp3
Partial name: e8807c2c-2cc3-4c12-a351-70a784ed5f36
  - joke1.1_e8807c2c-2cc3-4c12-a351-70a784ed5f36_1563717504810_417.mp3
  - joke1.1_e8807c2c-2cc3-4c12-a351-70a784ed5f36_1563717388349_887.mp3
Partial name: 0fa3f4b8-e2f6-44dc-a7a6-a47d49cdd546
  - joke1.1_0fa3f4b8-e2f6-44dc-a7a6-a47d49cdd546_1583361241622_731.mp3
  - joke1.1_0fa3f4b8-e2f6-44dc-a7a6-a47d49cdd546_1583361215310_662.mp3
Partial name: 22fa22ac-9f30-40e2-89b8-41ee3a8e89cf
  - joke1.1_22fa22ac-9f30-40e2-89b8-41ee3a8e89cf_1586730181497_917.mp3
  - joke1.1_22fa22ac-9f30-40e2-89b8-41ee3a8e89cf_1586730192425_68.mp3
Partial name: 60048bfe-321a-4d65-b84

In [50]:
def find_matches(partial_names, complete_names):
    data = []
    for partial in partial_names:
        matches = [name for name in complete_names if partial in name]
        if len(matches) == 1:
            data.append([partial, matches[0]])
    
    df = pd.DataFrame(data, columns=['VIDEOID/RESPONSEID', 'full_file_name'])
    return df

In [51]:
partial_complete_filenames = find_matches(labelled_joke1_1_partial_filename, filtered_files)

In [52]:
partial_complete_filenames.head()

,VIDEOID/RESPONSEID,full_file_name
0,73e855f3-6c3c-4673-86a0-c7f3dcfc7531,joke1.1_73e855f3-6c3c-4673-86a0-c7f3dcfc7531_1...
1,f889fba2-ad55-4b6a-ba67-baf589bf562e,joke1.1_f889fba2-ad55-4b6a-ba67-baf589bf562e_1...
2,ffd96b9b-89f7-4664-89a9-368ddea2e3e1,joke1.1_ffd96b9b-89f7-4664-89a9-368ddea2e3e1_1...
3,b1fbc9a4-f81f-4926-bc8f-c69b249d3259,joke1.1_b1fbc9a4-f81f-4926-bc8f-c69b249d3259_1...
4,b28fa101-b608-4c27-8f1e-dcf5cc8005b2,joke1.1_b28fa101-b608-4c27-8f1e-dcf5cc8005b2_1...


In [53]:
processed_results.rename(columns={"filename":"full_file_name"},inplace=True)

In [54]:
processed_results.head()

,full_file_name,num_laughs,results
0,joke1.1_088ca5ae-9b47-469b-a197-d7872f5ae9d6_1...,0,[]
1,joke1.1_5f9cb3c3-dba6-4a27-9881-57a95c7efe7a_1...,2,[{'file_name': './output_segments/joke1.1_5f9c...
2,joke1.1_22fa22ac-9f30-40e2-89b8-41ee3a8e89cf_1...,0,[]
3,joke1.1_2a3186e4-8f1e-42e8-84a3-172610e3dec4_1...,1,[{'file_name': './output_segments/joke1.1_2a31...
4,joke1.1_9f94b24b-25ac-4266-a3f5-17f48f7feaa6_1...,1,[{'file_name': './output_segments/joke1.1_9f94...


In [55]:
len(processed_results)

119

In [56]:
len(partial_complete_filenames)

70

In [59]:
# Combine the two DataFrames
combined_df = pd.concat([processed_results, partial_complete_filenames], ignore_index=True)
combined_df.head()

,full_file_name,num_laughs,results,VIDEOID/RESPONSEID
0,joke1.1_088ca5ae-9b47-469b-a197-d7872f5ae9d6_1...,0.0,[],NaN
1,joke1.1_5f9cb3c3-dba6-4a27-9881-57a95c7efe7a_1...,2.0,[{'file_name': './output_segments/joke1.1_5f9c...,NaN
2,joke1.1_22fa22ac-9f30-40e2-89b8-41ee3a8e89cf_1...,0.0,[],NaN
3,joke1.1_2a3186e4-8f1e-42e8-84a3-172610e3dec4_1...,1.0,[{'file_name': './output_segments/joke1.1_2a31...,NaN
4,joke1.1_9f94b24b-25ac-4266-a3f5-17f48f7feaa6_1...,1.0,[{'file_name': './output_segments/joke1.1_9f94...,NaN


In [61]:
# Drop all duplicate rows based on the 'full_file_name' column
unique_df = combined_df.drop_duplicates(subset='full_file_name', keep=False)
unique_df.head()

,full_file_name,num_laughs,results,VIDEOID/RESPONSEID
0,joke1.1_088ca5ae-9b47-469b-a197-d7872f5ae9d6_1...,0.0,[],NaN
2,joke1.1_22fa22ac-9f30-40e2-89b8-41ee3a8e89cf_1...,0.0,[],NaN
6,joke1.1_1319e4c3-e360-40f4-9279-fb1cfa9ed65a_1...,0.0,[],NaN
10,joke1.1_1319e4c3-e360-40f4-9279-fb1cfa9ed65a_1...,1.0,[{'file_name': './output_segments/joke1.1_1319...,NaN
11,joke1.1_60048bfe-321a-4d65-b842-5bea0ed98c58_1...,0.0,[],NaN


In [62]:
unique_df.shape

(53, 4)

In [63]:
# Join the dataframes using the 'key' column
joined_df = processed_results.merge(partial_complete_filenames, on='full_file_name', how='inner')

In [64]:

joined_df.shape

(70, 4)

In [65]:
joined_df.head()

,full_file_name,num_laughs,results,VIDEOID/RESPONSEID
0,joke1.1_5f9cb3c3-dba6-4a27-9881-57a95c7efe7a_1...,2,[{'file_name': './output_segments/joke1.1_5f9c...,5f9cb3c3-dba6-4a27-9881-57a95c7efe7a
1,joke1.1_2a3186e4-8f1e-42e8-84a3-172610e3dec4_1...,1,[{'file_name': './output_segments/joke1.1_2a31...,2a3186e4-8f1e-42e8-84a3-172610e3dec4
2,joke1.1_9f94b24b-25ac-4266-a3f5-17f48f7feaa6_1...,1,[{'file_name': './output_segments/joke1.1_9f94...,9f94b24b-25ac-4266-a3f5-17f48f7feaa6
3,joke1.1_246e8874-e4dd-4fbb-83ce-41d70ca77876_1...,4,[{'file_name': './output_segments/joke1.1_246e...,246e8874-e4dd-4fbb-83ce-41d70ca77876
4,joke1.1_246e8874-e4dd-4fbb-83ce-41d70ca77876_1...,4,[{'file_name': './output_segments/joke1.1_246e...,246e8874-e4dd-4fbb-83ce-41d70ca77876


In [66]:
joined_df.isnull().any(axis=1).any()

False

In [67]:
joined_df["VIDEOID/RESPONSEID"].nunique()

66

In [68]:
duplicate_rows = joined_df[joined_df["VIDEOID/RESPONSEID"].duplicated(keep=False)]
duplicate_rows

,full_file_name,num_laughs,results,VIDEOID/RESPONSEID
3,joke1.1_246e8874-e4dd-4fbb-83ce-41d70ca77876_1...,4,[{'file_name': './output_segments/joke1.1_246e...,246e8874-e4dd-4fbb-83ce-41d70ca77876
4,joke1.1_246e8874-e4dd-4fbb-83ce-41d70ca77876_1...,4,[{'file_name': './output_segments/joke1.1_246e...,246e8874-e4dd-4fbb-83ce-41d70ca77876
5,joke1.1_7d9817b4-3ee8-4cb3-a474-1789b08faf3a_1...,2,[{'file_name': './output_segments/joke1.1_7d98...,7d9817b4-3ee8-4cb3-a474-1789b08faf3a
6,joke1.1_7d9817b4-3ee8-4cb3-a474-1789b08faf3a_1...,2,[{'file_name': './output_segments/joke1.1_7d98...,7d9817b4-3ee8-4cb3-a474-1789b08faf3a
35,joke1.1_b28fa101-b608-4c27-8f1e-dcf5cc8005b2_1...,1,[{'file_name': './output_segments/joke1.1_b28f...,b28fa101-b608-4c27-8f1e-dcf5cc8005b2
36,joke1.1_b28fa101-b608-4c27-8f1e-dcf5cc8005b2_1...,1,[{'file_name': './output_segments/joke1.1_b28f...,b28fa101-b608-4c27-8f1e-dcf5cc8005b2
56,joke1.1_a33829cd-c0b8-4daf-9d2e-5689e9591413_1...,0,[],a33829cd-c0b8-4daf-9d2e-5689e9591413
57,joke1.1_a33829cd-c0b8-4daf-9d2e-5689e9591413_1...,0,[],a33829cd-c0b8-4daf-9d2e-5689e9591413


In [70]:
# duplicate free
unique_files = joined_df.drop_duplicates(subset="VIDEOID/RESPONSEID", keep=False)
unique_files.head()

,full_file_name,num_laughs,results,VIDEOID/RESPONSEID
0,joke1.1_5f9cb3c3-dba6-4a27-9881-57a95c7efe7a_1...,2,[{'file_name': './output_segments/joke1.1_5f9c...,5f9cb3c3-dba6-4a27-9881-57a95c7efe7a
1,joke1.1_2a3186e4-8f1e-42e8-84a3-172610e3dec4_1...,1,[{'file_name': './output_segments/joke1.1_2a31...,2a3186e4-8f1e-42e8-84a3-172610e3dec4
2,joke1.1_9f94b24b-25ac-4266-a3f5-17f48f7feaa6_1...,1,[{'file_name': './output_segments/joke1.1_9f94...,9f94b24b-25ac-4266-a3f5-17f48f7feaa6
7,joke1.1_c4f82db5-2182-47c3-8d85-ac0ca5b205c4_1...,1,[{'file_name': './output_segments/joke1.1_c4f8...,c4f82db5-2182-47c3-8d85-ac0ca5b205c4
8,joke1.1_e986fa51-78fe-4e4e-99cb-a7540bd83174_1...,1,[{'file_name': './output_segments/joke1.1_e986...,e986fa51-78fe-4e4e-99cb-a7540bd83174


In [72]:
unique_files.shape

(62, 4)

In [71]:
excel_file.head()

,VIDEOID/RESPONSEID,ChildHashedID,age_in_days,gender,Consent,JokeOrder,joke1.laugh.1,joke1.howfunny.1,joke1.laugh.2,joke1.howfunny.2,...,ThatsNotACat.laugh.2,ThatsNotACat.howfunny.2,ThatsNotACat.laugh.3,ThatsNotACat.howfunny.3,TearingPaper.laugh.1,TearingPaper.howfunny.1,TearingPaper.laugh.2,TearingPaper.howfunny.2,TearingPaper.laugh.3,TearingPaper.howfunny.3
0,4d16ddd9-e207-45d0-bed8-4e2bf8df60ea,FHN26F,105,f,accepted,"['ThatsNotACat', 'Peekaboo', 'ThatsNotAHat', '...",No,Not funny,No,Not funny,...,No,Not funny,No,Not funny,No,Slightly funny,No,Not funny,No,Not funny
1,73e855f3-6c3c-4673-86a0-c7f3dcfc7531,ZCH3UU,121,m,accepted,"['TearingPaper', 'NomNomNom', 'ThatsNotAHat', ...",No,Not funny,No,Not funny,...,No,Not funny,No,Not funny,No,Not funny,No,Not funny,No,Not funny
2,f889fba2-ad55-4b6a-ba67-baf589bf562e,USMEcW,132,f,accepted,"['TearingPaper', 'Peekaboo', 'ThatsNotAHat', '...",No,Not funny,No,Not funny,...,No,Not funny,No,Not funny,No,Not funny,No,Not funny,No,Not funny
3,ffd96b9b-89f7-4664-89a9-368ddea2e3e1,ZWYN3M,135,f,accepted,"['ThatsNotACat', 'NomNomNom', 'ThatsNotAHat', ...",No,Funny,No,Funny,...,No,Funny,No,Funny,No,Not funny,No,Not funny,No,Slightly funny
4,b1fbc9a4-f81f-4926-bc8f-c69b249d3259,PYDMU3,138,f,accepted,"['TearingPaper', 'ThatsNotAHat', 'Peekaboo', '...",No,Slightly funny,No,Not funny,...,No,Not funny,No,Slightly funny,No,Slightly funny,No,Not funny,Yes,Slightly funny


In [73]:
excel_file.shape

(94, 66)

In [74]:
# Join the dataframes using the 'key' column
final_combined = unique_files.merge(excel_file, on="VIDEOID/RESPONSEID", how='inner')
final_combined.shape

(62, 69)

In [75]:
final_combined.head()

,full_file_name,num_laughs,results,VIDEOID/RESPONSEID,ChildHashedID,age_in_days,gender,Consent,JokeOrder,joke1.laugh.1,...,ThatsNotACat.laugh.2,ThatsNotACat.howfunny.2,ThatsNotACat.laugh.3,ThatsNotACat.howfunny.3,TearingPaper.laugh.1,TearingPaper.howfunny.1,TearingPaper.laugh.2,TearingPaper.howfunny.2,TearingPaper.laugh.3,TearingPaper.howfunny.3
0,joke1.1_5f9cb3c3-dba6-4a27-9881-57a95c7efe7a_1...,2,[{'file_name': './output_segments/joke1.1_5f9c...,5f9cb3c3-dba6-4a27-9881-57a95c7efe7a,DXZ573,789,f,accepted,"['ThatsNotAHat', 'TearingPaper', 'Peekaboo', '...",No,...,Yes,Not funny,Yes,Slightly funny,No,Not funny,Yes,Slightly funny,Yes,Slightly funny
1,joke1.1_2a3186e4-8f1e-42e8-84a3-172610e3dec4_1...,1,[{'file_name': './output_segments/joke1.1_2a31...,2a3186e4-8f1e-42e8-84a3-172610e3dec4,6dZBcR,250,m,accepted,"['TearingPaper', 'ThatsNotAHat', 'NomNomNom', ...",No,...,No,Not funny,No,Not funny,No,Not funny,No,Slightly funny,No,Slightly funny
2,joke1.1_9f94b24b-25ac-4266-a3f5-17f48f7feaa6_1...,1,[{'file_name': './output_segments/joke1.1_9f94...,9f94b24b-25ac-4266-a3f5-17f48f7feaa6,RXFPN3,769,f,accepted,"['ThatsNotACat', 'ThatsNotAHat', 'NomNomNom', ...",No,...,No,Not funny,No,Not funny,Yes,Slightly funny,Yes,Slightly funny,Yes,Slightly funny
3,joke1.1_c4f82db5-2182-47c3-8d85-ac0ca5b205c4_1...,1,[{'file_name': './output_segments/joke1.1_c4f8...,c4f82db5-2182-47c3-8d85-ac0ca5b205c4,NSQQVX,469,m,accepted,"['ThatsNotACat', 'Peekaboo', 'TearingPaper', '...",No,...,Yes,Funny,Yes,Very funny,No,Slightly funny,Yes,Slightly funny,Yes,Slightly funny
4,joke1.1_e986fa51-78fe-4e4e-99cb-a7540bd83174_1...,1,[{'file_name': './output_segments/joke1.1_e986...,e986fa51-78fe-4e4e-99cb-a7540bd83174,AETYX2,327,m,accepted,"['ThatsNotAHat', 'NomNomNom', 'Peekaboo', 'Tea...",Yes,...,Yes,Slightly funny,No,Not funny,No,Not funny,No,Not funny,No,Not funny


In [76]:
final_combined["joke1.laugh.1"].value_counts()

No     40
Yes    22
Name: joke1.laugh.1, dtype: int64

In [78]:
final_combined['joke1.laugh.1_bool'] = final_combined['joke1.laugh.1'].map({'Yes': True, 'No': False})
final_combined.head()

,full_file_name,num_laughs,results,VIDEOID/RESPONSEID,ChildHashedID,age_in_days,gender,Consent,JokeOrder,joke1.laugh.1,...,ThatsNotACat.howfunny.2,ThatsNotACat.laugh.3,ThatsNotACat.howfunny.3,TearingPaper.laugh.1,TearingPaper.howfunny.1,TearingPaper.laugh.2,TearingPaper.howfunny.2,TearingPaper.laugh.3,TearingPaper.howfunny.3,joke1.laugh.1_bool
0,joke1.1_5f9cb3c3-dba6-4a27-9881-57a95c7efe7a_1...,2,[{'file_name': './output_segments/joke1.1_5f9c...,5f9cb3c3-dba6-4a27-9881-57a95c7efe7a,DXZ573,789,f,accepted,"['ThatsNotAHat', 'TearingPaper', 'Peekaboo', '...",No,...,Not funny,Yes,Slightly funny,No,Not funny,Yes,Slightly funny,Yes,Slightly funny,False
1,joke1.1_2a3186e4-8f1e-42e8-84a3-172610e3dec4_1...,1,[{'file_name': './output_segments/joke1.1_2a31...,2a3186e4-8f1e-42e8-84a3-172610e3dec4,6dZBcR,250,m,accepted,"['TearingPaper', 'ThatsNotAHat', 'NomNomNom', ...",No,...,Not funny,No,Not funny,No,Not funny,No,Slightly funny,No,Slightly funny,False
2,joke1.1_9f94b24b-25ac-4266-a3f5-17f48f7feaa6_1...,1,[{'file_name': './output_segments/joke1.1_9f94...,9f94b24b-25ac-4266-a3f5-17f48f7feaa6,RXFPN3,769,f,accepted,"['ThatsNotACat', 'ThatsNotAHat', 'NomNomNom', ...",No,...,Not funny,No,Not funny,Yes,Slightly funny,Yes,Slightly funny,Yes,Slightly funny,False
3,joke1.1_c4f82db5-2182-47c3-8d85-ac0ca5b205c4_1...,1,[{'file_name': './output_segments/joke1.1_c4f8...,c4f82db5-2182-47c3-8d85-ac0ca5b205c4,NSQQVX,469,m,accepted,"['ThatsNotACat', 'Peekaboo', 'TearingPaper', '...",No,...,Funny,Yes,Very funny,No,Slightly funny,Yes,Slightly funny,Yes,Slightly funny,False
4,joke1.1_e986fa51-78fe-4e4e-99cb-a7540bd83174_1...,1,[{'file_name': './output_segments/joke1.1_e986...,e986fa51-78fe-4e4e-99cb-a7540bd83174,AETYX2,327,m,accepted,"['ThatsNotAHat', 'NomNomNom', 'Peekaboo', 'Tea...",Yes,...,Slightly funny,No,Not funny,No,Not funny,No,Not funny,No,Not funny,True


In [80]:
final_combined[(final_combined["num_laughs"> 0]) & (final_combined["joke1.laugh.1_bool"]== True)].shape

TypeError: '>' not supported between instances of 'str' and 'int'

In [85]:
# Create the 'num_laughs_bool' column
final_combined['num_laughs_bool'] = final_combined['num_laughs'] > 0

In [28]:
final_combined=final_res

In [29]:
# Count where both columns have the same value
same_count = (final_combined['num_laughs_bool'] == final_combined['joke1.laugh.1_bool']).sum()

# Count where both columns have different values
different_count = (final_combined['num_laughs_bool'] != final_combined['joke1.laugh.1_bool']).sum()

print(f"Number of rows where both columns are the same: {same_count}")
print(f"Number of rows where both columns are different: {different_count}")

Number of rows where both columns are the same: 31
Number of rows where both columns are different: 31


In [91]:
num_files = same_count + different_count
num_files

62

In [93]:
accuracy = same_count/num_files
accuracy

0.7741935483870968

In [94]:
final_combined.shape

(62, 71)

In [95]:
final_combined.to_excel("laughted_detected.xlsx")

In [99]:
final_combined_reduced = final_combined[["full_file_name","num_laughs","results","VIDEOID/RESPONSEID","joke1.laugh.1","num_laughs_bool","joke1.laugh.1_bool"]]

In [102]:
final_combined_reduced.head(10)

,full_file_name,num_laughs,results,VIDEOID/RESPONSEID,joke1.laugh.1,num_laughs_bool,joke1.laugh.1_bool
0,joke1.1_5f9cb3c3-dba6-4a27-9881-57a95c7efe7a_1...,2,[{'file_name': './output_segments/joke1.1_5f9c...,5f9cb3c3-dba6-4a27-9881-57a95c7efe7a,No,True,False
1,joke1.1_2a3186e4-8f1e-42e8-84a3-172610e3dec4_1...,1,[{'file_name': './output_segments/joke1.1_2a31...,2a3186e4-8f1e-42e8-84a3-172610e3dec4,No,True,False
2,joke1.1_9f94b24b-25ac-4266-a3f5-17f48f7feaa6_1...,1,[{'file_name': './output_segments/joke1.1_9f94...,9f94b24b-25ac-4266-a3f5-17f48f7feaa6,No,True,False
3,joke1.1_c4f82db5-2182-47c3-8d85-ac0ca5b205c4_1...,1,[{'file_name': './output_segments/joke1.1_c4f8...,c4f82db5-2182-47c3-8d85-ac0ca5b205c4,No,True,False
4,joke1.1_e986fa51-78fe-4e4e-99cb-a7540bd83174_1...,1,[{'file_name': './output_segments/joke1.1_e986...,e986fa51-78fe-4e4e-99cb-a7540bd83174,Yes,True,True
5,joke1.1_f9ffff9f-7659-4b23-941e-988979f69ded_1...,1,[{'file_name': './output_segments/joke1.1_f9ff...,f9ffff9f-7659-4b23-941e-988979f69ded,No,True,False
6,joke1.1_5ce0c525-1675-4a81-a8ce-17adbd0fd06a_1...,1,[{'file_name': './output_segments/joke1.1_5ce0...,5ce0c525-1675-4a81-a8ce-17adbd0fd06a,No,True,False
7,joke1.1_178df8f7-68a6-464c-a172-4bbb6b05e01a_1...,1,[{'file_name': './output_segments/joke1.1_178d...,178df8f7-68a6-464c-a172-4bbb6b05e01a,Yes,True,True
8,joke1.1_3e424601-4435-4d85-8467-c800a67e3897_1...,1,[{'file_name': './output_segments/joke1.1_3e42...,3e424601-4435-4d85-8467-c800a67e3897,No,True,False
9,joke1.1_78490e71-df00-44cb-9eb7-56d6ad1f98fc_1...,2,[{'file_name': './output_segments/joke1.1_7849...,78490e71-df00-44cb-9eb7-56d6ad1f98fc,No,True,False


In [101]:
final_combined_reduced.to_excel("laugh_detection_labelled_results.xlsx")

In [1]:
import pandas as pd

In [2]:
final_res  = pd.read_excel("laugh_detection_labelled_results.xlsx")
final_res.head()

,Unnamed: 0,full_file_name,num_laughs,results,VIDEOID/RESPONSEID,joke1.laugh.1,num_laughs_bool,joke1.laugh.1_bool
0,0,joke1.1_5f9cb3c3-dba6-4a27-9881-57a95c7efe7a_1...,2,[{'file_name': './output_segments/joke1.1_5f9c...,5f9cb3c3-dba6-4a27-9881-57a95c7efe7a,No,True,False
1,1,joke1.1_2a3186e4-8f1e-42e8-84a3-172610e3dec4_1...,1,[{'file_name': './output_segments/joke1.1_2a31...,2a3186e4-8f1e-42e8-84a3-172610e3dec4,No,True,False
2,2,joke1.1_9f94b24b-25ac-4266-a3f5-17f48f7feaa6_1...,1,[{'file_name': './output_segments/joke1.1_9f94...,9f94b24b-25ac-4266-a3f5-17f48f7feaa6,No,True,False
3,3,joke1.1_c4f82db5-2182-47c3-8d85-ac0ca5b205c4_1...,1,[{'file_name': './output_segments/joke1.1_c4f8...,c4f82db5-2182-47c3-8d85-ac0ca5b205c4,No,True,False
4,4,joke1.1_e986fa51-78fe-4e4e-99cb-a7540bd83174_1...,1,[{'file_name': './output_segments/joke1.1_e986...,e986fa51-78fe-4e4e-99cb-a7540bd83174,Yes,True,True


In [3]:

final_res.num_laughs_bool.value_counts()

True     41
False    21
Name: num_laughs_bool, dtype: int64

In [4]:
final_res["joke1.laugh.1"].value_counts()

No     40
Yes    22
Name: joke1.laugh.1, dtype: int64

In [5]:
labelled_yes = final_res[final_res["joke1.laugh.1"] == "Yes"]

In [6]:
labelled_yes.shape

(22, 8)

In [7]:
labelled_yes["num_laughs_bool"].value_counts()

True     16
False     6
Name: num_laughs_bool, dtype: int64

In [32]:
labelled_no = final_res[final_res["joke1.laugh.1"] == "No"]
labelled_no.shape 

(40, 9)

In [33]:
labelled_no["joke1.laugh.1"].value_counts()

No    40
Name: joke1.laugh.1, dtype: int64

In [34]:
labelled_no["num_laughs_bool"].value_counts()

True     25
False    15
Name: num_laughs_bool, dtype: int64

In [15]:
labelled_no.tail(10)

,Unnamed: 0,full_file_name,num_laughs,results,VIDEOID/RESPONSEID,joke1.laugh.1,num_laughs_bool,joke1.laugh.1_bool
46,46,joke1.1_cf9963f0-eaf3-46ad-b553-fab4e9a24503_1...,0,[],cf9963f0-eaf3-46ad-b553-fab4e9a24503,No,False,False
47,47,joke1.1_7aeaa6c5-7941-4d55-b29d-852255290f84_1...,0,[],7aeaa6c5-7941-4d55-b29d-852255290f84,No,False,False
50,50,joke1.1_b1fbc9a4-f81f-4926-bc8f-c69b249d3259_1...,0,[],b1fbc9a4-f81f-4926-bc8f-c69b249d3259,No,False,False
51,51,joke1.1_4901fe93-aaa8-4557-9717-34c9c845e4af_1...,2,[{'file_name': './output_segments/joke1.1_4901...,4901fe93-aaa8-4557-9717-34c9c845e4af,No,True,False
53,53,joke1.1_079bdab2-71e0-4969-b222-c3d606a31701_1...,0,[],079bdab2-71e0-4969-b222-c3d606a31701,No,False,False
54,54,joke1.1_fe12cbe7-bf8b-40a0-aff6-ffdfa29af575_1...,0,[],fe12cbe7-bf8b-40a0-aff6-ffdfa29af575,No,False,False
55,55,joke1.1_8a85d911-0df9-412d-ace0-d3901cada917_1...,1,[{'file_name': './output_segments/joke1.1_8a85...,8a85d911-0df9-412d-ace0-d3901cada917,No,True,False
56,56,joke1.1_e813c4c8-0ab1-4812-8e33-de2e05aa610c_1...,0,[],e813c4c8-0ab1-4812-8e33-de2e05aa610c,No,False,False
57,57,joke1.1_0c6a9044-d414-41fc-ad1c-0ea3535bdd41_1...,0,[],0c6a9044-d414-41fc-ad1c-0ea3535bdd41,No,False,False
61,61,joke1.1_40729233-7572-49cc-b765-38eb8f541372_1...,0,[],40729233-7572-49cc-b765-38eb8f541372,No,False,False


In [14]:
labelled_no.iloc[0]["full_file_name"]

'joke1.1_5f9cb3c3-dba6-4a27-9881-57a95c7efe7a_1587767083654_972.mp3'

In [20]:
final_res["match"] = final_res["joke1.laugh.1_bool"] == final_res["num_laughs_bool"] 
final_res.head()

,Unnamed: 0,full_file_name,num_laughs,results,VIDEOID/RESPONSEID,joke1.laugh.1,num_laughs_bool,joke1.laugh.1_bool,match
0,0,joke1.1_5f9cb3c3-dba6-4a27-9881-57a95c7efe7a_1...,2,[{'file_name': './output_segments/joke1.1_5f9c...,5f9cb3c3-dba6-4a27-9881-57a95c7efe7a,No,True,False,False
1,1,joke1.1_2a3186e4-8f1e-42e8-84a3-172610e3dec4_1...,1,[{'file_name': './output_segments/joke1.1_2a31...,2a3186e4-8f1e-42e8-84a3-172610e3dec4,No,True,False,False
2,2,joke1.1_9f94b24b-25ac-4266-a3f5-17f48f7feaa6_1...,1,[{'file_name': './output_segments/joke1.1_9f94...,9f94b24b-25ac-4266-a3f5-17f48f7feaa6,No,True,False,False
3,3,joke1.1_c4f82db5-2182-47c3-8d85-ac0ca5b205c4_1...,1,[{'file_name': './output_segments/joke1.1_c4f8...,c4f82db5-2182-47c3-8d85-ac0ca5b205c4,No,True,False,False
4,4,joke1.1_e986fa51-78fe-4e4e-99cb-a7540bd83174_1...,1,[{'file_name': './output_segments/joke1.1_e986...,e986fa51-78fe-4e4e-99cb-a7540bd83174,Yes,True,True,True


In [24]:
final_res.tail(10)

,Unnamed: 0,full_file_name,num_laughs,results,VIDEOID/RESPONSEID,joke1.laugh.1,num_laughs_bool,joke1.laugh.1_bool,match
52,52,joke1.1_8cdac953-fe67-44d1-a875-77c9eae75984_1...,1,[{'file_name': './output_segments/joke1.1_8cda...,8cdac953-fe67-44d1-a875-77c9eae75984,Yes,True,True,True
53,53,joke1.1_079bdab2-71e0-4969-b222-c3d606a31701_1...,0,[],079bdab2-71e0-4969-b222-c3d606a31701,No,False,False,True
54,54,joke1.1_fe12cbe7-bf8b-40a0-aff6-ffdfa29af575_1...,0,[],fe12cbe7-bf8b-40a0-aff6-ffdfa29af575,No,False,False,True
55,55,joke1.1_8a85d911-0df9-412d-ace0-d3901cada917_1...,1,[{'file_name': './output_segments/joke1.1_8a85...,8a85d911-0df9-412d-ace0-d3901cada917,No,True,False,False
56,56,joke1.1_e813c4c8-0ab1-4812-8e33-de2e05aa610c_1...,0,[],e813c4c8-0ab1-4812-8e33-de2e05aa610c,No,False,False,True
57,57,joke1.1_0c6a9044-d414-41fc-ad1c-0ea3535bdd41_1...,0,[],0c6a9044-d414-41fc-ad1c-0ea3535bdd41,No,False,False,True
58,58,joke1.1_f4e620b4-8cf1-485a-961b-3a0828415ec5_1...,1,[{'file_name': './output_segments/joke1.1_f4e6...,f4e620b4-8cf1-485a-961b-3a0828415ec5,Yes,True,True,True
59,59,joke1.1_60083074-de80-48f1-899a-437c3952ae59_1...,0,[],60083074-de80-48f1-899a-437c3952ae59,Yes,False,True,False
60,60,joke1.1_164e487b-e4b6-4cb0-bb05-56a1f65e8324_1...,1,[{'file_name': './output_segments/joke1.1_164e...,164e487b-e4b6-4cb0-bb05-56a1f65e8324,Yes,True,True,True
61,61,joke1.1_40729233-7572-49cc-b765-38eb8f541372_1...,0,[],40729233-7572-49cc-b765-38eb8f541372,No,False,False,True


In [23]:
final_res.match.value_counts()

False    31
True     31
Name: match, dtype: int64

In [25]:
final_res.dtypes

Unnamed: 0             int64
full_file_name        object
num_laughs             int64
results               object
VIDEOID/RESPONSEID    object
joke1.laugh.1         object
num_laughs_bool         bool
joke1.laugh.1_bool      bool
match                   bool
dtype: object

In [26]:
final_res.full_file_name.nunique()

62

In [30]:
final_res.head()


,Unnamed: 0,full_file_name,num_laughs,results,VIDEOID/RESPONSEID,joke1.laugh.1,num_laughs_bool,joke1.laugh.1_bool,match
0,0,joke1.1_5f9cb3c3-dba6-4a27-9881-57a95c7efe7a_1...,2,[{'file_name': './output_segments/joke1.1_5f9c...,5f9cb3c3-dba6-4a27-9881-57a95c7efe7a,No,True,False,False
1,1,joke1.1_2a3186e4-8f1e-42e8-84a3-172610e3dec4_1...,1,[{'file_name': './output_segments/joke1.1_2a31...,2a3186e4-8f1e-42e8-84a3-172610e3dec4,No,True,False,False
2,2,joke1.1_9f94b24b-25ac-4266-a3f5-17f48f7feaa6_1...,1,[{'file_name': './output_segments/joke1.1_9f94...,9f94b24b-25ac-4266-a3f5-17f48f7feaa6,No,True,False,False
3,3,joke1.1_c4f82db5-2182-47c3-8d85-ac0ca5b205c4_1...,1,[{'file_name': './output_segments/joke1.1_c4f8...,c4f82db5-2182-47c3-8d85-ac0ca5b205c4,No,True,False,False
4,4,joke1.1_e986fa51-78fe-4e4e-99cb-a7540bd83174_1...,1,[{'file_name': './output_segments/joke1.1_e986...,e986fa51-78fe-4e4e-99cb-a7540bd83174,Yes,True,True,True


In [31]:
final_res.to_excel("final_results.xlsx",index=False)

In [ ]:
final_res